In [2]:
import ipdb
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pylab as pylab
import cartopy
from utils import constants as cnst
from eod import msg
from utils import u_grid, u_interpolate as u_int, constants as cnst, u_arrays, u_darrays, u_met
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as feature
import cartopy.io.shapereader as shapereader
import pickle as pkl
import glob

coast = shapereader.natural_earth(resolution='110m',
                                  category='physical',
                                  name='coastline')

countries = shapereader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [3]:
filetag = 'pl'
era_hour = 12

In [4]:
#node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node3_MCS_matchTable.p', "rb"))
node_days = pd.read_csv(cnst.DATA + '/SOMS/saves/3x3_Nodes_fullYear/mcs_matchtable/node3_MCS_matchTable_mergedNodes_5mmMCS_fullYear.csv', index_col=0, parse_dates=True)

In [5]:
pd.to_datetime(node_days.index)[0].replace(hour=5)

Timestamp('2004-04-09 05:30:00')

In [27]:
def save_node_clim_old(filetag, node, era_hour):
    
    nodestr = 'node'+str(node)
    node_days = pd.read_csv(cnst.DATA + '/SOMS/saves/3x3_Nodes_fullYear/mcs_matchtable/'+nodestr+'_MCS_matchTable_mergedNodes_5mmMCS_fullYear.csv', index_col=0, parse_dates=True)
    #node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/3x3/MCStable_3x3_5-9N/'+nodestr+'_MCS_matchTable.p', "rb"))
    
    #### potentially reduce MCS box here
    region = (node_days.lat>5) & (node_days.lat<8) & (node_days.lon>-10) & (node_days.lon<10)
    node_days = node_days[region]
    
    node_days_era = []
    for nd in node_days.index:
        node_days_era.append(nd.replace(hour=era_hour))
    node_days_era = pd.Series(node_days_era).values
        
    if filetag == 'pl':
        fft = 'pressure_levels'
    else:
        fft = 'surface'
        
    filebase = cnst.other_drive + '/ERA5_WAf/hourly/' 
    filepath = filebase + fft+'/ERA5*'+filetag+'.nc'
    out = cnst.DATA + 'SOMS/saves/3x3_Nodes_fullYear/'+nodestr+'_ERA5_5MCS_5mm_mean_5-8N_old_'+filetag+'.nc'
    outdev = cnst.DATA + 'SOMS/saves/3x3_Nodes_fullYear/'+nodestr+'_ERA5_5MCS_5mm_stddev_5-8N_old_'+filetag+'.nc'
    
    def means(da, node_days):
        da = da.sel(time=(da['time.hour']==era_hour))
        try:
            da = da.sel(level=[925,850,750,650,550,250])
        except:
            pass
        
        if (filetag == 'pl'):
            if not 'z' in da.data_vars:
                ipdb.set_trace()
                
        dal = []
        
        u, inv = np.unique(node_days, return_inverse=True)
        n = np.bincount(inv)
        goodinds = u[(n>=5)] #minimum 5 MCS per MCS day
        
        intersect_dates = np.intersect1d(goodinds, da.time.values)
        if len(intersect_dates) == 0:
            return None
        try:
            da = da.sel(time=intersect_dates)
        except:
            return None

        da = u_darrays.flip_lat(da)
        return da
    
    srfc_list = []
    
    for f in glob.glob(filepath):
        print('Doing', f)
        srfc = xr.open_dataset(f) 
        srfc_concat = means(srfc,node_days_era)
        if srfc_concat is None:
            continue

        srfc_list.append(srfc_concat)
    
    full = xr.concat(srfc_list, dim='time')
    #ipdb.set_trace()
    mean = full.mean('time')
    dev = full.std('time')
    
    mean.to_netcdf(out)
    dev.to_netcdf(outdev)

In [6]:
def save_node_clim(filetag, node, era_hour):
    
    nodestr = 'node'+str(node)
    node_days = pd.read_csv(cnst.DATA + '/SOMS/saves/3x3_Nodes_fullYear/mcs_matchtable/'+nodestr+'_MCS_matchTable_mergedNodes_5mmMCS_fullYear.csv', index_col=0, parse_dates=True)
    #node_days = pkl.load(open(cnst.network_data + 'data/SOMS/saves/3x3/MCStable_3x3_5-9N/'+nodestr+'_MCS_matchTable.p', "rb"))
    
    #### potentially reduce MCS box here
    region = (node_days.lat>5) & (node_days.lat<8) & (node_days.lon>-10) & (node_days.lon<10)
    node_days = node_days[region]
    
#     node_days_era = []
#     for nd in node_days.index:
#         node_days_era.append(nd.replace(hour=era_hour))
#     node_days_era = pd.Series(node_days_era).values

    node_days_era = []
    for nd in node_days.index:
        node_days_era.append(nd)
    node_days_era = pd.Series(node_days_era).values
        
    if filetag == 'pl':
        fft = 'pressure_levels'
    else:
        fft = 'surface'
        
    filebase = cnst.other_drive + '/ERA5_WAf/hourly/' 
    filepath = filebase + fft+'/ERA5*'+filetag+'.nc'
    out = cnst.DATA + 'SOMS/saves/3x3_Nodes_fullYear/'+nodestr+'_ERA5_5MCS_5mm_mean_5-8N_5MCS_'+filetag+'.nc'
    outdev = cnst.DATA + 'SOMS/saves/3x3_Nodes_fullYear/'+nodestr+'_ERA5_5MCS_5mm_stddev_5-8N_5MCS_'+filetag+'.nc'
    
    def means(da, node_days):
        da = da.sel(time=(da['time.hour']==era_hour))
        try:
            da = da.sel(level=[925,850,750,650,550,250])
        except:
            pass
        
        if (filetag == 'pl'):
            if not 'z' in da.data_vars:
                ipdb.set_trace()
                
        dal = []

        u, inv = np.unique(node_days, return_inverse=True)
        n = np.bincount(inv)
        goodinds = u[(n>=3)] #minimum 5 MCS per MCS day (5 in one hour)
        goodinds = pd.Series(goodinds)
        goodins_erahour = []
   
        for nd in goodinds:
            goodins_erahour.append(nd.replace(hour=era_hour, minute=0))
        
        goodins_erahour = np.unique(goodins_erahour)
        goodins_erahour = pd.Series(goodins_erahour).values
        
        #ipdb.set_trace()
        intersect_dates = np.intersect1d(goodins_erahour, da.time.values)
        
        if len(intersect_dates) == 0:
            return None
        try:
            da = da.sel(time=intersect_dates)
        except:
            return None

        da = u_darrays.flip_lat(da)
        return da
    
    srfc_list = []
    
    for f in glob.glob(filepath):
        #print('Doing', f)
        srfc = xr.open_dataset(f) 
        srfc_concat = means(srfc,node_days_era)
        if srfc_concat is None:
            continue

        srfc_list.append(srfc_concat)
    
    print(node, 'saved MCS days nb', len(srfc_list), len(node_days), len(srfc_list)/len(node_days)*100)
    full = xr.concat(srfc_list, dim='time')
    #ipdb.set_trace()
    mean = full.mean('time')
    dev = full.std('time')
    
#     mean.to_netcdf(out)
#     dev.to_netcdf(outdev)

In [7]:
for ni in range(1,10):
#     nodestr='node'+str(1)
#     all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/node_days.p', "rb"))
#     node_days = all_node_file[nodestr]
#     node_days_era = []
#     for nd in node_days:
#         node_days_era.append(nd.replace(hour=era_hour))
        
    save_node_clim(filetag, ni, era_hour)

1 saved MCS days nb 61 2225 2.741573033707865
2 saved MCS days nb 65 2322 2.7993109388458226
3 saved MCS days nb 56 1751 3.198172472872644
4 saved MCS days nb 56 1270 4.409448818897638
5 saved MCS days nb 59 1865 3.1635388739946384
6 saved MCS days nb 56 1763 3.176403857061826
7 saved MCS days nb 23 647 3.554868624420402
8 saved MCS days nb 41 1291 3.1758326878388843
9 saved MCS days nb 54 1866 2.8938906752411575


In [ ]:
f = plt.figure(figsize=(9,5), dpi=200)

toconcat = []
for node in range(1,10):
    toconcat.append(pd.read_csv(cnst.DATA + '/SOMS/saves/3x3_Nodes_fullYear/mcs_matchtable/node'+str(node)+'_MCS_matchTable_mergedNodes_5mmMCS_fullYear.csv', index_col=0, parse_dates=True))
    
df = pd.concat(toconcat)
df = df.where(df.hour==18).dropna()
    
nmonth = []
ax = f.add_subplot(1,1,1)

mdays = [31,28,31,30,31,30,31,31,30,31,30,31]
for ni in range(1,13):
    nmonth.append(np.sum(df.month==ni)/(mdays[ni-1]*np.unique(df.year).size)) # note, divided by number of days in years 2004-2015! 
    print(np.unique(df.year))
print(nmonth)
ax.bar(np.arange(1,13).astype(int), np.array(nmonth))
ax.set_ylabel('MCS per day [considering MCS days only]')
ax.set_xlabel('Months')
#ax.set_title('Node:'+ str(node)+' '+merged_nodes_str[node-1])
ax.set_xticks(np.arange(1,13).astype(int))
    
plt.tight_layout()
#f.savefig(cnst.network_data + 'figs/SOMS/nb_node_permonth_nodes_fullYear.jpg')

In [ ]:
all_node_file = pkl.load(open(cnst.network_data + 'data/SOMS/saves/3x3_Nodes_fullYear/node_days.p', "rb"))

In [ ]:
(all_node_file['node1']).size

In [ ]:
from utils import u_darrays as uda
f = plt.figure(figsize=(15,5), dpi=300)
mname = 'node'

for ids, m in enumerate(range(1,10)):
    
    
    pl_clim = xr.open_dataset(cnst.DATA + 'SOMS/saves/3x3_Nodes_fullYear/node1_ERA5_clim_mean_srfc.nc')
    #pl_clim = uda.flip_lat(pl_clim)
    pl_clim = pl_clim['cape'].sel(latitude=slice(3,10), longitude=slice(-12,12))
    pl_clim.values = pl_clim.values*0
    
    df = pd.read_csv(cnst.DATA + '/SOMS/saves/3x3_Nodes_fullYear/mcs_matchtable/node'+str(m)+'_MCS_matchTable_mergedNodes_5mmMCS_fullYear.csv', index_col=0, parse_dates=True)
    df = df.where(df.hour==18).dropna()
    anf = all_node_file['node'+str(m)]
    #ipdb.set_trace()
    anf = anf.where((anf.year>=2004)&(anf.year<=2015)).dropna()
    
    daycount = df.index.size/anf.size #*100 
    #ipdb.set_trace()

    for ix, dff in df.iterrows():

        xv = dff.lon
        yv = dff.lat

        loc = pl_clim.sel(longitude=xv, latitude=yv, method='nearest')
        xm, ym = np.meshgrid(pl_clim.longitude.values, pl_clim.latitude.values)
        ipos = np.where((xm==loc.longitude.values)&(ym==loc.latitude.values))
        try:
            pl_clim.values[ipos] = pl_clim.values[ipos]+1
        except:
            ipdb.set_trace()

    plots = pl_clim.values #/ anf.size *100
    
    ax = f.add_subplot(3,3,ids+1, projection=ccrs.PlateCarree())

    plt.contourf(pl_clim.longitude, pl_clim.latitude, plots, transform=ccrs.PlateCarree(), cmap='PuBuGn', extend='both', levels=np.arange(0,5))
    ax.coastlines()
    # Gridlines
    xl = ax.gridlines(draw_labels=True);
    xl.top_labels = False
    xl.right_labels = False
    # Countries
    ax.add_feature(cartopy.feature.BORDERS, linestyle='--');
    plt.title('MCS/day: '+str(np.round(daycount,1)))
    cbar = plt.colorbar()
    cbar.set_label('Total MCS (n)')
   
    
    
plt.tight_layout()
#f.savefig(cnst.network_data + 'figs/LSTA/corrected_LSTA/new/wavelet_coefficients/2hOverlap/maps.png')
        
        